In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import math

opts = webdriver.ChromeOptions()
opts.add_experimental_option('prefs', {
    'download.default_directory': r"C:\Users\USER\Downloads"
})
driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe", options=opts)

In [ ]:
url = "https://movie.daum.net/moviedb/grade?movieId=##MOVIENO##""

driver.get(url)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

comment_num_txt = soup.find('span', {'class': 'txt_netizen'}) #총 data 수
comment_num = int(comment_num_txt.text[1:-2])

# 클릭 수 = (총 data 수 - 10) / 30(클릭 당 새로 호출되는 data 수)
click_num = (comment_num - 10) / 30
click_num

In [ ]:
# 데이터 최신 순으로 조회
new_button = '//*[@class="link_cate #latest"]'
driver.find_element('xpath', new_button).click()

# fetch more
for i in tqdm(range(math.ceil(click_num))):
    more_button = '//*[@class="link_fold #more"]'
    driver.find_element('xpath', more_button).click()
    time.sleep(2) 

In [ ]:
# data 가져오기
html = driver.page_source  #현재 페이지 정보 갱신
soup = BeautifulSoup(html, 'html.parser')

tmp_list = []   # data 저장 리스트
review = soup.find_all('p', {'class': 'desc_txt font_size_'})

for p in review:
    ptext = p.text
    a = ptext.replace('\n', '') 
    tmp_list.append(a.strip())

# 앞 3 데이터 check  
tmp_list[0:3]

In [ ]:
len(tmp_list)

In [ ]:
# s_data 가져오기(data 있다면)

comments = soup.find("ul", {"class": "list_comment"}).find_all("li")
star_list = []

for abc in comments:
    if abc.find('p', {'class': 'desc_txt font_size_'}):
        ans = abc.find_all("div")[1]
        star_list.append(ans.text.strip())
        
star_list[0:3]

In [ ]:
len(star_list)

In [ ]:
# star_list에서의 pos/neg data 인덱스
pos_index = []
neg_index = []

for i in range(len(star_list)):
    if int(star_list[i]) < 8:
        neg_index.append(i)
    else:
        pos_index.append(i)
        
# pos/neg data 리스트 생성
pos_list = []
neg_list = []

for i in range(len(pos_index)):
    pos_list.append(tmp_list[pos_index[i]])
    
for i in range(len(neg_index)):
    neg_list.append(tmp_list[neg_index[i]])

In [ ]:
# analyze
from konlpy.tag import Okt  # pip install konlpy
import time

start = time.time()
okt = Okt()

# 전체 data 분석
all_doc = '\n'.join(tmp_list)
all_doc_noun = okt.nouns(all_doc)
print(time.time() - start)

In [ ]:
start = time.time()
okt = Okt()

# pos data 분석
pos_doc = '\n'.join(pos_list)
pos_doc_noun = okt.nouns(pos_doc)
print(time.time() - start)

In [ ]:
start = time.time()
okt = Okt()

# neg data 분석
neg_doc = '\n'.join(neg_list)
neg_doc_noun = okt.nouns(neg_doc)
print(time.time() - start)

In [ ]:
from collections import Counter
from wordcloud import WordCloud  # pip install wordcloud
import matplotlib.pyplot as plt
import numpy as np 

count_noun = Counter(pos_doc_noun)

stopword_list = ['DEFAULT']

for word in count_noun:
    if len(word) == 1:
        stopword_list.append(word)
        
stopword_list[-5:]

for stopword in stopword_list:
    if stopword in count_noun:
        count_noun.pop(stopword)

wc_noun = WordCloud(background_color='white', max_words=2000, font_path=r'C:\Windows\Fonts\H2MJRE.ttf')
wc_noun = wc_noun.generate_from_frequencies(count_noun)

plt.imshow(wc_noun, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
count_noun = Counter(neg_doc_noun)

stopword_list = ['DEFAULT']

for word in count_noun:
    if len(word) == 1:
        stopword_list.append(word)
        
stopword_list[-5:]

for stopword in stopword_list:
    if stopword in count_noun:
        count_noun.pop(stopword)

wc_noun = WordCloud(background_color='white', max_words=2000, font_path=r'C:\Windows\Fonts\H2MJRE.ttf')
wc_noun = wc_noun.generate_from_frequencies(count_noun)

plt.imshow(wc_noun, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
import pandas as pd

star_posneg = []

for i in range(len(star_list)):
    if int(star_list[i]) < 8:
        star_posneg.append(0)
    else:
        star_posneg.append(1) 

# dataframe 생성
df = pd.DataFrame({'id': 0, 'data': tmp_list, 'ratings': star_posneg})
df

In [ ]:
# txt파일 생성
df.to_csv('data1.txt', sep = '\t', index = False)

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')
import urllib.request 

def load_data(file_path):
    train = []
    count = 0
    with open(file_path, 'r', encoding = 'utf-8') as f:
        for line in f.readlines():
            if count == 3000: break
            line = line.strip()
            id, doc, label = line.split('\t')
            if label == '1': label = 'pos'
            elif label == '0': label = 'neg'
            train.append((doc, label))
            count += 1
            
    return train

In [ ]:
train = load_data('data1.txt')
train = train[1:]
print(train[0:3])

In [ ]:
def pos_tokenize(raw_sent):
    pos_sent =[]
    
    sent = okt.pos(raw_sent, norm=True, stem=True)
    
    for tup in sent:
        word, tag = tup[0], tup[1]
        word_tag = word + '/' + tag
        pos_sent.append(word_tag)
        
    return ' '.join(pos_sent)

def make_word_dict(train, use_morph=False):
    all_words = set()

    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        for word in words:
            all_words.add(word)
            
    return all_words


def make_train_feats(train, all_words, use_morph=False):
    train_features = []

    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        tmp = {set_word: (set_word in words) for set_word in all_words}
        sent_tup = (tmp, label)
        train_features.append(sent_tup)
        
    return train_features

In [ ]:
use_morph = True
all_words = make_word_dict(train, use_morph)
print('word 집합 개수: ', len(all_words))
train_features = make_train_feats(train, all_words, use_morph)

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

In [ ]:
class MyNaiveBayesClassifier:
    def __init__(self, k=0.5, use_morph=False):
        self.k = k
        self.word_probs = []
        self.use_morph = use_morph
        
        if self.use_morph:
            from konlpy.tag import Okt
            self.okt = Okt()
            
    def load_data(self, file_path):
        docs = []
        labels = []
        
        count = 0
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f.readlines():
                if count == 500:
                    break
                line = line.strip()
                id, doc, label = line.split('\t')
                docs.append(doc)
                if label == '1': label = 'pos'
                elif label == '0': label = 'neg'
                labels.append(label)
                count += 1
            
        return docs[1:], labels[1:]
    
    def tokenize(self, sentence):
        if self.use_morph:
            pos_sent = []
            
            sent = self.okt.pos(sentence, norm=True, stem=True)
            
            for tup in sent:
                word, tag = tup
                word_tag = word + '/' + tag
                pos_sent.append(word_tag)
                
            sentence = ' '.join(pos_sent)
            
        return sentence.split()
    
    def count_words(self, docs, labels):
        count_dict = dict()
        for doc, label in zip(docs, labels):
            for word in self.tokenize(doc):
                if word not in count_dict:
                    count_dict[word] = {'pos': 0, 'neg':0}
                count_dict[word][label] += 1
        
        print('num of words...', len(count_dict))
        return count_dict
    
    def word_prob(self, count_dict, pos_class_num, neg_class_num, k):
        word_prob_list = []
        
        for word in count_dict:
            pos_word_num = count_dict[word]['pos']
            neg_word_num = count_dict[word]['neg']
            
            pos_class_prob = (pos_word_num + k) / (pos_class_num + 2*k)
            neg_class_prob = (neg_word_num + k) / (neg_class_num + 2*k)
            
            tup = (word, pos_class_prob, neg_class_prob)
            word_prob_list.append(tup)
        
        self.word_prob_list = word_prob_list
        return word_prob_list
    
    def class_prob(self, word_prob_list, test_sentence, use_unseen=False):
        test_words = self.tokenize(test_sentence)
        
        sent_log_pos_class_prob, sent_log_neg_class_prob = 0.0, 0.0
        
        for word, word_pos_class_prob, word_neg_class_prob in word_prob_list:
            if word in test_words:
                sent_log_pos_class_prob += math.log(word_pos_class_prob) 
                sent_log_neg_class_prob += math.log(word_neg_class_prob)
            else:
                if use_unseen:
                    sent_log_pos_class_prob += math.log(1-word_pos_class_prob) 
                    sent_log_neg_class_prob += math.log(1-word_neg_class_prob)
            
        sent_pos_class_prob = math.exp(sent_log_pos_class_prob)
        sent_neg_class_prob = math.exp(sent_log_neg_class_prob)
        
        pos_class_prob = sent_pos_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        neg_class_prob = sent_neg_class_prob/(sent_pos_class_prob+sent_neg_class_prob)
        
        return pos_class_prob, neg_class_prob
    
    def train(self, train_file_path):
        train_docs, train_labels = self.load_data(train_file_path)
        
        word_count_dict = self.count_words(train_docs, train_labels)
        
        pos_class_num = len([label for label in train_labels if label == 'pos'])
        neg_class_num = len([label for label in train_labels if label == 'neg'])
        
        self.word_probs = self.word_prob(word_count_dict, pos_class_num, neg_class_num, self.k)

    def classify(self, test_sentence, use_unseen=False):
        pos_class_prob, neg_class_prob = self.class_prob(self.word_prob_list, test_sentence, use_unseen)
        
        if pos_class_prob > neg_class_prob:
            print('pos', pos_class_prob)
        else:
            print('neg', neg_class_prob)

In [ ]:
classifier = MyNaiveBayesClassifier(use_morph=True)

In [ ]:
classifier.train('data1.txt')

In [ ]:
classifier.classify('SAMPLE TXT GOES HERE', use_unseen=True)

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='Malgun Gothic')

star_int_list = []
tempo = 0

for i in range(len(star_list)):
    tempo += int(star_list[i])
    star_int_list.append(tempo/ (i+1))
    
reverse_list = star_int_list[::-1]

plt.rcParams['axes.unicode_minus'] = False
plt.title('data 변동성')
plt.plot(reverse_list, color='crimson', label='data1')
plt.xlabel('data 수')
plt.ylabel('ratings AVG')
plt.legend(loc=4)
plt.show()